In [10]:
import pandas as pd
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

columns = ['Source', 'Target']
df = pd.read_csv('/Users/hunjunsin/Desktop/Jun/Unsupervised/hw6/edges_sampled_2K.csv', names=columns)

#create graph

G = ig.Graph()
G.add_vertices(list(map(str, set(df['Source']).union(set(df['Target'])))))

edges = list(zip(df['Source'], df['Target']))
edges = [(str(source), str(target)) for source, target in edges]
G.add_edges(edges)


In [11]:
c=G.components()
for i in c:
    print(i)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 142, 143, 144, 145, 146, 147, 149, 151, 152, 153, 154, 156, 157, 158, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 194, 195, 196, 198, 199, 200, 201, 202, 203, 204, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 220, 222, 223, 224, 226, 227, 229, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 251, 

In [12]:
def compute_modularity(graph, membership):
    
    if isinstance(membership, list) and all(isinstance(comm, list) for comm in membership):
        membership_list = [-1] * graph.vcount()  # 초기화
        for i, community in enumerate(membership):
            for node_idx in community:
                membership_list[node_idx] = i
        return graph.modularity(membership_list)
    else:
        return graph.modularity(membership)


Girvan Newman Algorithm

In [13]:
def girvan_newman_max_modularity(graph, max_iter=None):

    working_graph = graph.copy()
    
    # 커뮤니티와 모듈성 이력 추적
    community_history = []
    modularity_history = []
    
    # 초기 상태 추가 (연결 컴포넌트)
    components = working_graph.components()
    community_history.append(components)
    
    # 멤버십 리스트 형식을 사용하여 모듈성 계산
    modularity_history.append(compute_modularity(working_graph, components.membership))
    
    iteration = 0
    while working_graph.ecount() > 0:
        if max_iter is not None and iteration >= max_iter:
            break
            
        # Compute Edge betweenness 
        edge_betweenness = working_graph.edge_betweenness()
        
        # find the edge with highest betweenness
        max_betweenness_index = edge_betweenness.index(max(edge_betweenness))
        
        # delete the edge
        working_graph.delete_edges(max_betweenness_index)
        
        # find the connected components
        components = working_graph.components()
        
        # 커뮤니티 상태와 모듈성 추적
        community_history.append(components)
        current_modularity = compute_modularity(working_graph, components.membership)
        modularity_history.append(current_modularity)
        
        iteration += 1
    
    # find the best modularity
    best_idx = np.argmax(modularity_history)
    best_communities = community_history[best_idx]
    best_modularity = modularity_history[best_idx]
    
    # 최적의 커뮤니티 구조를 vertex 이름으로 변환
    best_communities_named = []
    for community in best_communities:
        community_named = [working_graph.vs[vertex]['name'] for vertex in community]
        best_communities_named.append(community_named)
    
    return best_communities_named, best_modularity, modularity_history, best_idx, community_history

In [ ]:
print("Running Girvan-Newman algorithm to maximize modularity...")
communities, modularity, modularity_history, best_iter, community_history = girvan_newman_max_modularity(G, max_iter=10)
community_sizes = [len(community) for community in communities]

print("\n===== comparison with built in library =====")
built_in_communities = G.community_edge_betweenness()
built_in_clusters = built_in_communities.as_clustering(n=None)  

built_in_result = []
for community in built_in_clusters:
    community_named = [G.vs[vertex]['name'] for vertex in community]
    built_in_result.append(community_named)

# compare the number of communities
print(f"Number of communities (custom implementation): {len(communities)}")
print(f"Number of communities (built-in implementation): {len(built_in_result)}")


node_to_community = {}
for i, community in enumerate(communities):
    for node in community:
        node_to_community[node] = i

custom_membership = [-1] * G.vcount()  # 초기화
for i in range(G.vcount()):
    node_name = G.vs[i]['name']
    if node_name in node_to_community:
        custom_membership[i] = node_to_community[node_name]

custom_modularity = G.modularity(custom_membership)
built_in_modularity = built_in_clusters.modularity

print(f"\nModularity score (custom implementation): {custom_modularity:.4f}")
print(f"Modularity score (built-in implementation): {built_in_modularity:.4f}")

custom_sizes = sorted([len(comm) for comm in communities], reverse=True)
built_in_sizes = sorted([len(comm) for comm in built_in_result], reverse=True)

print(f"\nCommunity sizes (custom): {custom_sizes}")
print(f"Community sizes (built-in): {built_in_sizes}")

community_counts = [len(community_history[i]) for i in range(len(modularity_history))]


Running Girvan-Newman algorithm to maximize modularity...

===== comparison with built in library =====
Number of communities (custom implementation): 14
Number of communities (built-in implementation): 10

Modularity score (custom implementation): 0.4142
Modularity score (built-in implementation): 0.4147

Community sizes (custom): [168, 89, 34, 8, 4, 3, 2, 2, 2, 2, 1, 1, 1, 1]
Community sizes (built-in): [172, 89, 34, 8, 4, 3, 2, 2, 2, 2]
